Optimizers

Gradient Descent
- Stochastic Gradient Descent - fitting single sample at a time
- vanilla gradient descent, gradient descent, batch gradient descent - fitting whole dataset at once
- mini batch dataset - fit smaller (mini) batches of data instead of all data at once
- These terms can get confusing. For the purpose of the book we will call mini batches batches 
- some call it stochastic gradient descent regardless of batch size/single sample

- to implement gradient descent, need a learing rate and the calculated gradients of loss function with respect to parameters. To get the parameter update amounts just multiply -learning rate * gradients, then add to the parameters. Learning rate is negated because we are trying to find minimum, so stepping towards lowest loss
- see optimizer object below - in previous work we stored the layer weights and biases gradients in the layer objects as attributes so we can now make use of & modify them via the optimzer

In [1]:
class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1.0):
        self.learning_rate = learning_rate
    # Update parameters
    def update_params(self, layer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases

Basic network training using SGD optimizer
- epoch - a full pass through the training data, including forwards and backwards
- typically models are trained over multiple epochs, but obviously the less the better
- here it is implemented via for loop where each epoch is 1 interation of the loop, including a forward and backward pass, so do that 10000x
- for this intial run, we chose a learning rate of 1, (it is the default)
- run the final cell in this workbook with full network code to use this example

In [10]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD()
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)

epoch: 0, acc: 0.360, loss: 1.099
epoch: 100, acc: 0.400, loss: 1.087
epoch: 200, acc: 0.417, loss: 1.077
epoch: 300, acc: 0.413, loss: 1.076
epoch: 400, acc: 0.400, loss: 1.074
epoch: 500, acc: 0.403, loss: 1.071
epoch: 600, acc: 0.417, loss: 1.067
epoch: 700, acc: 0.440, loss: 1.062
epoch: 800, acc: 0.457, loss: 1.055
epoch: 900, acc: 0.410, loss: 1.062
epoch: 1000, acc: 0.407, loss: 1.058
epoch: 1100, acc: 0.407, loss: 1.057
epoch: 1200, acc: 0.403, loss: 1.064
epoch: 1300, acc: 0.427, loss: 1.051
epoch: 1400, acc: 0.443, loss: 1.067
epoch: 1500, acc: 0.400, loss: 1.058
epoch: 1600, acc: 0.420, loss: 1.070
epoch: 1700, acc: 0.410, loss: 1.049
epoch: 1800, acc: 0.460, loss: 1.040
epoch: 1900, acc: 0.483, loss: 1.033
epoch: 2000, acc: 0.403, loss: 1.038
epoch: 2100, acc: 0.447, loss: 1.022
epoch: 2200, acc: 0.467, loss: 1.023
epoch: 2300, acc: 0.437, loss: 1.005
epoch: 2400, acc: 0.497, loss: 0.993
epoch: 2500, acc: 0.513, loss: 0.981
epoch: 2600, acc: 0.453, loss: 0.991
epoch: 2700, 

- the output of this intial training method showed some learing (accuracy ~63%), but loss did not drop 
- the visualiztion of this training: https://nnfs.io/pup
- in the visualization, there is a "flashy wiggle" effect, indicating the learning rate may be too high. We can also tell via the loss, which does not decrease smoothly, but bounces around, (i.e. the loss decreases between epochs, then increases again between other epochs)

The Learning Rate
- in most cases applying the full negative gradient to the parameters is too big of a step, as the gradient is continously changing (and we are applying a point estimate of a tangent function). So a big jump may result in jumping over the steepest areas of the function instead of more closely hugging to the function's curvature.
- small steps make sure we follow the direction of steepest descent, hugging more closely to the function curve by not jumping too much. But too small is bad too - takes longer to arive at optimal parameters and more prone to getting stuck in local minimums, the lowest point of a function in a given x range (vs global minimum - the lowest possible y value a function can output)
- ideal global minimum for ANNs is a loss of 0 - however this is typically not achieved. You know you are stuck in some local minumum if the loss is not low/close to 0
- Gradient descent algo follows the direction of steepest descent, no matter how large or small it is. So if you are near a local minimum, this is what causes it to get stcuk because graidents near local minimum are lower, causing smaller parameter adjustment
- too low learning rate can cause learning stagnation - stuck in local minimum
- too high learning rate can cause gradient explosion - where gradient updates cause model loss to rise instead of fall. Eventually loss/gradients become so big that they cannot be stored in floating point, causing error. Can be costly if model has taken a while to train, then waste of time and computing resources
- with just learning rate, need to select an learning rate that is not too high or too low for the reasons discussed above. Too high and will cause loss to bounce around/gradient explosion, too low and the model will learn too slow.
- alternatives to just setting learning rate are learning rate decay and momentum
- setting these factors are called hyper parameters, and setting them appropriately requires experience, it is difficult to prescribe anything. Have to see how model performs with different hyperparameters and adjust from there.
- see code example below for lower learning rate - BE SURE TO RUN FINAL CELL SO THE OBJECTS WORK
- in the code example, setting learning rate to .85 causes slightly higher accuracy, and slightly lower loss

In [12]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(learning_rate=.85)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)

epoch: 0, acc: 0.360, loss: 1.099
epoch: 100, acc: 0.403, loss: 1.091
epoch: 200, acc: 0.410, loss: 1.078
epoch: 300, acc: 0.423, loss: 1.077
epoch: 400, acc: 0.413, loss: 1.075
epoch: 500, acc: 0.400, loss: 1.074
epoch: 600, acc: 0.410, loss: 1.071
epoch: 700, acc: 0.417, loss: 1.067
epoch: 800, acc: 0.440, loss: 1.064
epoch: 900, acc: 0.443, loss: 1.057
epoch: 1000, acc: 0.420, loss: 1.050
epoch: 1100, acc: 0.397, loss: 1.061
epoch: 1200, acc: 0.387, loss: 1.060
epoch: 1300, acc: 0.420, loss: 1.061
epoch: 1400, acc: 0.460, loss: 1.055
epoch: 1500, acc: 0.390, loss: 1.057
epoch: 1600, acc: 0.450, loss: 1.072
epoch: 1700, acc: 0.400, loss: 1.049
epoch: 1800, acc: 0.423, loss: 1.039
epoch: 1900, acc: 0.387, loss: 1.059
epoch: 2000, acc: 0.437, loss: 1.053
epoch: 2100, acc: 0.443, loss: 1.026
epoch: 2200, acc: 0.377, loss: 1.050
epoch: 2300, acc: 0.433, loss: 1.016
epoch: 2400, acc: 0.460, loss: 1.000
epoch: 2500, acc: 0.493, loss: 1.010
epoch: 2600, acc: 0.527, loss: 0.998
epoch: 2700, 

Learning Rate Decay
 - general idea is the start with some learning rate at beginning of training and decrease it during training
 - One way - monitor loss across each epoch and adjust learning rate if there is learning stagnation (ie. loss curve is flattening, rate too low) or jumping in loss (rate too high). Can be programmed or just done manually
 - Can also use a decay rate that decreases the learning rate per batch or per epoch, such as a 1/t decay or exponential decay.
 - we will use learning rate decay, specifically: starting_learning rate * (1 / (1+ learning_rate_decay * step_number)). So the larger the step number (scaled by the decay factor) the lower the learning rate. This form also ensure that we do not accidently increase learning rate by divinding by a value less than 1, hence the adding 1.
 - See below for new optimizer class with learning rate decay
 - decay is is defualt 0, and if decay is not 0, then we update it with the steps
 - we keep track of iterations, though not sure if it is necessary if epoch/batches are via range function. I guess it makes it more self-contained, and if you are not looping with numbers then it would be good to internally count iterations.


In [17]:
class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
        layer.weights += -self.current_learning_rate * layer.dweights
        layer.biases += -self.current_learning_rate * layer.dbiases
    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

Traning with Learning Rate
- trial and error - learning rate decay of 1e-2 was too high, that is the rate dropped too fast and the model got stuck in local minimum. (higher decay causes larger denominator because we multiply decay by step size), 1e-3 is better, getting to our best accuracy/loss so far. This assumes learning rate of 1.
- be sure to run optimzier object cell above and last cell in notebook with all prerequisite code

In [18]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(decay=1e-3)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099
epoch: 100, acc: 0.400, loss: 1.088
epoch: 200, acc: 0.423, loss: 1.078
epoch: 300, acc: 0.423, loss: 1.076
epoch: 400, acc: 0.420, loss: 1.076
epoch: 500, acc: 0.403, loss: 1.074
epoch: 600, acc: 0.403, loss: 1.072
epoch: 700, acc: 0.410, loss: 1.070
epoch: 800, acc: 0.410, loss: 1.068
epoch: 900, acc: 0.427, loss: 1.066
epoch: 1000, acc: 0.440, loss: 1.063
epoch: 1100, acc: 0.440, loss: 1.059
epoch: 1200, acc: 0.447, loss: 1.056
epoch: 1300, acc: 0.440, loss: 1.052
epoch: 1400, acc: 0.427, loss: 1.048
epoch: 1500, acc: 0.417, loss: 1.040
epoch: 1600, acc: 0.423, loss: 1.033
epoch: 1700, acc: 0.450, loss: 1.025
epoch: 1800, acc: 0.470, loss: 1.017
epoch: 1900, acc: 0.460, loss: 1.008
epoch: 2000, acc: 0.463, loss: 1.000
epoch: 2100, acc: 0.490, loss: 1.005
epoch: 2200, acc: 0.467, loss: 1.014
epoch: 2300, acc: 0.483, loss: 1.014
epoch: 2400, acc: 0.490, loss: 1.012
epoch: 2500, acc: 0.493, loss: 1.009
epoch: 2600, acc: 0.497, loss: 1.005
epoch: 2700, 

Momentum
- momentum creates a rolling average of gradients over some number of epochs or training steps
- helps to avoid getting stuck in local minimums because if previous gradients were very high, they will play into the current weight update, which might be small because the model is near a local minimum. So the momentum will increase the weight update preventing the model from getting stuck in the local minimum
- kind of like a ball rolling down a hill, the steeper the hill, the bigger the hill on the other side you need to slow it down
- in other words, the momentum helps you stay on the path of global gradient descent -looking at the greater overall average gradient - than just the gradient that is what is right next to you.
- weight_updates_with_momentum = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights; where self.momentum is a momentum hyperparameter between 0 and 1 and weight momentums is an array of previous weight momentums, consisting of the previous weight updates: layer.weight_momentums = weight_updates, instantiated at 0s.
- so basically the momentum is the previous update to the parameters, then we just incorporate new gradient info by subtracting the current gradient times the learning rate. The greater the momentum parameter, the greater the weight on the previous gradients * learning rates
- thinking through, if momentum parameter is > 0 and the previous step was a very high gradient, then the next step is in the same direction, then the gradient update will increase. If it is in the opposite then it will decrease, but be greater than it would've been without momentum. This is where the momentum prevents the falling into local minimum. With each new step, there is greater emphasis on more recent gradient updates and the other update values are overwritten by the rolling sum, and more so if momentum param < 1. This is an exponential moving average

New Update Paras function for Optimizer SGD
- checks if momentum is not 0, if not
- check if we have the attribute weight momentums (ie, is this the first epoch/step), and if not, then makes a gradient of 0s
- performs the formula described above, and then sets the weight momentums to that value for the next epoch when the new weights are calculated.
- then, updates weights like done previously 

In [ ]:
def update_params(self, layer):
# If we use momentum
    if self.momentum:
    # If layer does not contain momentum arrays, create them
    # filled with zeros
        if not hasattr(layer, 'weight_momentums'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            # If there is no momentum array for weights
            # The array doesn't exist for biases yet either.
            layer.bias_momentums = np.zeros_like(layer.biases)
        
        # Build weight updates with momentum - take previous
        # updates multiplied by retain factor and update with
        # current gradients
        weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
        layer.weight_momentums = weight_updates
        # Build bias updates
        bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
        layer.bias_momentums = bias_updates

    else:
        weight_updates = -self.current_learning_rate * layer.dweights
        bias_updates = -self.current_learning_rate * layer.dbiases

    layer.weights += weight_updates
    layer.biases += bias_updates

Full Optimizer With Momentum

In [23]:
class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
    # If we use momentum
        if self.momentum:
        # If layer does not contain momentum arrays, create them
        # filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                # If there is no momentum array for weights
                # The array doesn't exist for biases yet either.
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            # Build bias updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

Training with Momentum
- be sure to run cell above for optimizer class with momentum AND run the last cell with the other network class
- performance with momentum =.9 is the best we've seen so far => accuracy 93.3% and 

In [28]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(decay=1e-3, momentum=.9)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}, ' +
        f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.443, loss: 1.053, lr: 0.9099181073703367
epoch: 200, acc: 0.497, loss: 0.999, lr: 0.8340283569641367
epoch: 300, acc: 0.603, loss: 0.810, lr: 0.7698229407236336
epoch: 400, acc: 0.700, loss: 0.700, lr: 0.7147962830593281
epoch: 500, acc: 0.750, loss: 0.595, lr: 0.66711140760507
epoch: 600, acc: 0.810, loss: 0.496, lr: 0.6253908692933083
epoch: 700, acc: 0.810, loss: 0.466, lr: 0.5885815185403178
epoch: 800, acc: 0.847, loss: 0.384, lr: 0.5558643690939411
epoch: 900, acc: 0.850, loss: 0.364, lr: 0.526592943654555
epoch: 1000, acc: 0.877, loss: 0.344, lr: 0.5002501250625312
epoch: 1100, acc: 0.863, loss: 0.347, lr: 0.4764173415912339
epoch: 1200, acc: 0.880, loss: 0.322, lr: 0.45475216007276037
epoch: 1300, acc: 0.887, loss: 0.309, lr: 0.43497172683775553
epoch: 1400, acc: 0.877, loss: 0.295, lr: 0.4168403501458941
epoch: 1500, acc: 0.887, loss: 0.277, lr: 0.4001600640256102
epoch: 1600, acc: 0.893, loss: 0.270, lr: 0.38476337

AdaGrad - Adaptive Gradient
- Adaptive gradient institutes a per-parameter learning rate rather than a globally shared rate.
- When do you use adagrad? When you have sparse gradients (i.e., many gradient values are 0) or variables with different sizes/magnitudes. It may also be more computationally efficient than an optimizer like adam. However, adagrad is not used as frequently as adam
- adagrad formula: cache += parm_gradient ** 2; parm_updates = learning_rate * parm_gradient /(sqrt(cache) + eps)
- so keep a cumulative sum of squared gradients for a parameter in the cache variable, then divide the current parameter gradient by the square root of the cache value plus epsilon
- this has the effect of progressively lowering the gradient update amount for parameters that have very high gradients 
- epsilon is a hyperparameter preventing division by 0. Typically it is set very low, such as 1e-7
- summing the squared values and then taking the square root causes the overall parm_gradient to be scaled more slowly -> 1 + 3 =4; sqrt(1^2 + 3^2) = 2.16
- so basically gradient updates get progressively smaller if cache is rising. Parameters with small gradients will have learning rates decreased slower than ones with higher 
- Per questioning chatGPT, this basically assumes that frequently updated (i.e. non sparse) or large parameters have less of an impact or include less new information for a constant change in learning rate than would a smaller/sparse parameter for the same change in learning rate.
- adagrad is prone to stalling because if the cache becomes very large, then updates will be very small
- see below for new update params function 

In [ ]:
def update_params(self, layer):
    # If layer does not contain cache arrays,
    # create them filled with zeros
    if not hasattr(layer, 'weight_cache'):
        layer.weight_cache = np.zeros_like(layer.weights)
        layer.bias_cache = np.zeros_like(layer.biases)
    # Update cache with squared current gradients
    layer.weight_cache += layer.dweights**2
    layer.bias_cache += layer.dbiases**2
    # Vanilla SGD parameter update + normalization
    # with square rooted cache
    layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
    layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

Full adagrad function

In [4]:
class Optimizer_Adagrad:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

Testing out Adagrad
- note please run the cell above and the last cell in workbook when testing
- with decay of 1e-4, this does pretty well but not as good as SGD with momentum, 1e-4 works best for learning rate decay

In [5]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_Adagrad(decay=1e-4)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}, ' +
        f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.457, loss: 1.012, lr: 0.9901970492127933
epoch: 200, acc: 0.527, loss: 0.936, lr: 0.9804882831650161
epoch: 300, acc: 0.600, loss: 0.874, lr: 0.9709680551509855
epoch: 400, acc: 0.623, loss: 0.830, lr: 0.9616309260505818
epoch: 500, acc: 0.617, loss: 0.791, lr: 0.9524716639679969
epoch: 600, acc: 0.663, loss: 0.761, lr: 0.9434852344560807
epoch: 700, acc: 0.667, loss: 0.733, lr: 0.9346667912889054
epoch: 800, acc: 0.680, loss: 0.704, lr: 0.9260116677470135
epoch: 900, acc: 0.687, loss: 0.686, lr: 0.9175153683824203
epoch: 1000, acc: 0.683, loss: 0.669, lr: 0.9091735612328392
epoch: 1100, acc: 0.703, loss: 0.653, lr: 0.9009820704567978
epoch: 1200, acc: 0.700, loss: 0.640, lr: 0.892936869363336
epoch: 1300, acc: 0.717, loss: 0.629, lr: 0.8850340738118416
epoch: 1400, acc: 0.713, loss: 0.615, lr: 0.8772699359592947
epoch: 1500, acc: 0.727, loss: 0.601, lr: 0.8696408383337683
epoch: 1600, acc: 0.737, loss: 0.589, lr: 0.86214328

RMSProp
- similar to adagrad in that we are incorporating per-parameter learing rate, however, in this case it is done via an exponentially weighted moving average similar to how momentum is incorporated
- RMSProp formula: cache = rho * cache + (1 - rho) * gradient ** 2; as with adagrad, use the sqrt of cache to plus epsilon to scale the gradient
- benefits of this approach over adagrad are that the learning rate can change upwards or downwards depending on the most recent gradients, as opposed to adagrad where the cache keeps increasing with each non-zero gradient. Also, does not accumulate very old gradients and can control how much emphasis on old gradients there is via rho.
- so get to track the global direction of descent better and makes it less prone learning stalling
- rho parameter adjusts empasis on recent gradients vs older gradients. The higher rho is, the smaller portion each new gradient has in the cache, and the more influence older gradients have. Greater rho = greater memory of old gradients
- need to lower learning rate as well => my guess is because a higher learning rate would result in very large gradients if somehow the cache became very low or near zero. Dividing by a very small number would create big gradient update possibly causing instability. 

RMSProp Class
- exact same as adagrad, just changed the cache funtions and learning rate. Note that instead of just doing +=, the moving average is now setting the cache variable equal to the previous cache and current gradients weighted by rho or 1-rho

In [9]:
class Optimizer_RMSprop:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=.001, decay=0., epsilon=1e-7, rho=.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        # Update cache with squared current gradients
        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases**2
        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

Testing out RMSProp
- note please run the cell above and the last cell in workbook when testing
- still not as good as SGD with momentum

In [10]:
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

###NOTE: run full network code in final cell of this workbook so that this example works

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_RMSprop(learning_rate=0.02, decay=1e-5, rho=0.999)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    if not epoch % 100: #only every hundredth epoch
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, ' +
        f'loss: {loss:.3f}, ' +
        f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099, lr: 0.02
epoch: 100, acc: 0.467, loss: 1.014, lr: 0.01998021958261321
epoch: 200, acc: 0.530, loss: 0.959, lr: 0.019960279044701046
epoch: 300, acc: 0.563, loss: 0.891, lr: 0.019940378268975763
epoch: 400, acc: 0.587, loss: 0.839, lr: 0.01992051713662487
epoch: 500, acc: 0.593, loss: 0.805, lr: 0.01990069552930875
epoch: 600, acc: 0.623, loss: 0.762, lr: 0.019880913329158343
epoch: 700, acc: 0.653, loss: 0.750, lr: 0.019861170418772778
epoch: 800, acc: 0.597, loss: 0.897, lr: 0.019841466681217078
epoch: 900, acc: 0.670, loss: 0.736, lr: 0.01982180200001982
epoch: 1000, acc: 0.710, loss: 0.634, lr: 0.019802176259170884
epoch: 1100, acc: 0.767, loss: 0.612, lr: 0.01978258934311912
epoch: 1200, acc: 0.760, loss: 0.566, lr: 0.01976304113677013
epoch: 1300, acc: 0.733, loss: 0.607, lr: 0.019743531525483964
epoch: 1400, acc: 0.777, loss: 0.537, lr: 0.01972406039507293
epoch: 1500, acc: 0.797, loss: 0.528, lr: 0.019704627631799327
epoch: 1600, acc: 0.777, lo

Adam Optimizer
- is RMSprop with momentums added back in
- before getting in to adam fully, a few key points and insights missed on the optimizers from above: 
1) the momentum is the exponentially weighted average of the gradients, also called first moment - so we are using momentum to tell us the average direction of the gradients. It helps smooth out the noise in the gradients and get us out of local minimums
2) the cache is the exponentially weighted uncenterd variance of the gradients, also called the second moment - (i.e. we are taking the squared deviation - not subtracting anything because assuming mean is 0, and not dividing because don't divide in EWMA). A note on uncentered variance - i feel like it is assuming the mean of the gradient is 0, but chatgpt is adament this is not an assumption in the context of Adam. Anyway, dividing by the sqrt of the cache causes gradients with higher variance to take smaller steps, and those with lower variance to take larger steps. High variance indicates an unsmooth and potentially unstable gradien, so smaller steps should be taken to not overshoot the minimum. Conversely, small variance indicates a smooth gradient and can proceed more confidently with larger steps, speeding up convergence.
3) the momentum cache is initialzed as zeros, see RMS prop code or adam code below. This is done because it is a simple, "neutral" way to get a starting value for the mean gradients, again has assuming 0 all over it, at least assuming 0 to start. However, the true means of the gradients may be different. So when we begin to calculate EWMA, it will be biased towards 0 because of the 0 intialization.
4) since we use a 0 intialized momentums and cache, we have this bias towards 0 in early steps of training because there is not enough info present to estimate a true mean, which may slow down convergence or cause instability in optimization.

- So now adam - in our implementation, we include the momentum and adaptive learning rate (scaling by the sqrt of second moment see #2 above) to help with the learing process, and we also attempt to correct the 0 bias. We use two hyperparameters to do this: Beta1 and Beta2, using the weights formula as example
1) Momentum - Uses Beta1. self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights. So beta1 is the weight placed on the historical gradients, and 1-beta1 is the weight on current gradient
2) Uncentered Variance - Uses Beta2. layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2. So beta2 is how much empahsis on historical variance factors s 1- beta2 for current variance
3) To adjust the means and variances upward, we divide them by 1 - beta (their respective betas), so adjusted momentum = momentum / (1 - beta1); adjusted variance = variance / (1 - beta2). Why do we use the same parameter? If you work out the math: EWMAt = EWMAt-1 * Beta + (1 - Beta) * Current Weights, so Current Weights = (EWMAt - EWMAt-1)/(1-Beta). So basically we divide by 1 - Beta because [Finish investigating this!]
4) point about beta approaching 0 with each step to progressively remove normalization

Full Network Code from Ch. 9 So We can make use of it for examples above

In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()
import matplotlib.pyplot as plt

# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from input ones, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
        # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

# ReLU activation
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Since we need to modify original variable,
        # let’s make a copy of values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax activation
class Activation_Softmax:
# Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities
    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)
        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

# Common loss class
class Loss:
    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):
        
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        
        # Return loss
        return data_loss
    
class Loss_CategoricalCrossentropy(Loss):
# Forward pass
    def forward(self, y_pred, y_true):
    # Number of samples in a batch
        samples = len(y_pred)
        
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])
        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples


class Activation_Softmax_Loss_CategoricalCrossentropy():
# Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
        # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)

        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
    
        # For each row in dinputs, get what the network has for the correct class and subtract 1
        self.dinputs[range(samples), y_true] -= 1
        
        # Normalize gradient
        self.dinputs = self.dinputs / samples